# Vectorizer + NaiveBayes Tuning

🎯 The goal of this challenge is to create a Pipeline combining a Vectorizer + a NaiveBayes algorithm and to fine-tune the pipeline.

✍️ Let's reuse the previous dataset with $2000$ reviews classified either as "positive" or "negative".

In [1]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data["target_encoded"] =  le.fit_transform(data.target)

In [3]:
data.head()

,target,reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...",0
1,neg,the happy bastard's quick movie review \ndamn ...,0
2,neg,it is movies like these that make a jaded movi...,0
3,neg,""" quest for camelot "" is warner bros . ' firs...",0
4,neg,synopsis : a mentally unstable man undergoing ...,0


## Preprocessing

❓ **Question (Cleaning)** ❓

Clean your texts

In [4]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    sentence = sentence.lower()
    
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') 
    
    sentence = sentence.strip()
    
    word_tokens = word_tokenize(sentence)
    
    
    lemmatize_str = [WordNetLemmatizer().lemmatize(word, pos='v') 
                               for word in word_tokens]
   
    return ' '.join(lemmatize_str)

In [5]:
# Clean reviews
data['cleaned_reviews'] = pd.DataFrame([preprocessing(sentence) for sentence in data['reviews']])
data.head()

,target,reviews,target_encoded,cleaned_reviews
0,neg,"plot : two teen couples go to a church party ,...",0,plot two teen couple go to a church party drin...
1,neg,the happy bastard's quick movie review \ndamn ...,0,the happy bastards quick movie review damn tha...
2,neg,it is movies like these that make a jaded movi...,0,it be movies like these that make a jade movie...
3,neg,""" quest for camelot "" is warner bros . ' firs...",0,quest for camelot be warner bros first feature...
4,neg,synopsis : a mentally unstable man undergoing ...,0,synopsis a mentally unstable man undergo psych...


## Tuning

❓ **Question (Pipelining a Vectorizer and a NaiveBayes Model)** ❓

* Create a Pipeline that chains a vectorizer of your choice with a NaiveBayes model
* Optimize it
* What is your best estimator ?

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import set_config; set_config("diagram")
from sklearn.pipeline import make_pipeline

# Create Pipeline
pipeline_naive_bayes = make_pipeline( 
            TfidfVectorizer(),
            MultinomialNB()
)

# Set parameters to search
# Define the grid of parameters
parameters = {
    'tfidfvectorizer__ngram_range': ((1,1), (2,2)),
    'multinomialnb__alpha': (0.1,1)
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline_naive_bayes,
    parameters,
    scoring = "recall",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

# Perform grid search on pipeline
grid_search.fit(data.cleaned_reviews,data.target_encoded)

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score = 0.8959999999999999
Best params = {'multinomialnb__alpha': 1, 'tfidfvectorizer__ngram_range': (2, 2)}


In [15]:
from sklearn.model_selection import cross_validate

# Tuned Pipeline
tuned_pipeline_naive_bayes = make_pipeline( 
            TfidfVectorizer(ngram_range = (2,2)),
            MultinomialNB(alpha=1)
)

cv_result = cross_validate(tuned_pipeline_naive_bayes, data.cleaned_reviews, data.target_encoded, cv = 5, scoring=['recall'])
cv_result['test_recall'].mean()

0.8959999999999999

🏁 Congratulations! You've managed to chain a Vectorizer and a NLP model and fine-tuned it!

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!